## Sampler for myocardium

#### Read composite then output at the same base_dir and visualize

In [23]:
import os 
from glob import glob
import shutil
from PIL import Image
import numpy as np

input_dir = 'D:\Data\CNNTrainingCrop64_Added'
composite_data_total = glob(os.path.join(input_dir, 'CompositeLabelTotal', '*'))
output_dir = os.path.join(input_dir, 'LabelMyoTotal')
output_dir_vis = os.path.join(input_dir, 'LabelMyoTotal_Visualize')
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

if not os.path.exists(output_dir_vis):
    os.mkdir(output_dir_vis)
    
for label in composite_data_total:
    fname = label.split('\\')[-1]
    composite = np.array(Image.open(label))
    myocardium_bol = (composite == 2) | (composite == 4)
    myocardium = np.zeros(myocardium_bol.shape)
    myocardium[myocardium_bol == True] = 2
    myocardium[myocardium_bol == False] = 1
    
    myocardium_visualize = np.zeros(myocardium_bol.shape)
    myocardium_visualize[myocardium_bol == True] = 255
    myocardium_visualize[myocardium_bol == False] = 0 
    
    im_myo = Image.fromarray(myocardium)
    im_myo.save(os.path.join(output_dir, fname))
    
    im_myo_vis = Image.fromarray(myocardium_visualize)
    im_myo_vis.save(os.path.join(output_dir_vis, fname))

## Copy TrainingDataTotal and LabelDataTotal to input

In [24]:
import os 
from glob import glob
import shutil

input_dir = 'D:\Data\CNNTrainingCrop64_Added'
train_data_total = glob(os.path.join(input_dir, 'TrainingDataTotal', '*'))
label_data_total = glob(os.path.join(input_dir, 'LabelMyoTotal', '*'))

output_dir = 'D:\DeepLearn\input'
output_dir_train = os.path.join(output_dir, 'TrainingDataTotal')
output_dir_label = os.path.join(output_dir, 'LabelDataTotal')

for train in train_data_total:
    fname = train.split('\\')[-1]
    shutil.copyfile(train, os.path.join(output_dir_train, fname))
    
for label in label_data_total:
    fname = label.split('\\')[-1]
    shutil.copyfile(label, os.path.join(output_dir_label, fname))  

## Remove unqualified images based on QCToExclude

In [25]:
from scipy.io import loadmat
x = loadmat(os.path.join(input_dir, 'QCToExclude.mat'))
exclude_array = x['exclude_array']

flat_array = [item for sublist in exclude_array for item in sublist]
numToExclude = sum(flat_array)

## Random sampling

In [26]:
import numpy as np
import random
train_glob = glob(os.path.join(output_dir_train, '*'))
fnames = [train.split('\\')[-1] for train in train_glob]
flat_array = [item for sublist in exclude_array for item in sublist]
ex_list = []
for i, e in enumerate(flat_array):
    if e == 1:
        print(fnames[i])
        ex_list.append(fnames[i])
        
for ex in ex_list:
    fnames.remove(ex)
    
if len(fnames)%5 != 0:
    fnum = int(np.ceil(len(fnames)/5) * 5)
else:
    fnum = len(fnames)


training_data = random.sample(fnames, int(fnum*0.8))
training_set = set(training_data)
total_set = set(fnames)
testing_set = total_set - training_set
testing_data = list(testing_set)

136.tif
137.tif


## Copy data to each directory

In [27]:
for train in training_data:
    shutil.copyfile(os.path.join(os.path.join(output_dir, 'TrainingDataTotal', train)), os.path.join(output_dir, 'TrainingData',train))
    shutil.copyfile(os.path.join(os.path.join(output_dir, 'LabelDataTotal', train)), os.path.join(output_dir, 'TrainingLabel',train))
    
for test in testing_data:
    shutil.copyfile(os.path.join(os.path.join(output_dir, 'TrainingDataTotal', test)), os.path.join(output_dir, 'TestingData',test))
    shutil.copyfile(os.path.join(os.path.join(output_dir, 'LabelDataTotal', test)), os.path.join(output_dir, 'TestingLabel',test))